In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# 使用JRC数据集做交叉验证

## JRC

### 分三类验证

In [ ]:
# 使用JRC做验证
JRC_2018 = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum().rename('jrcwaterclass')
JRC_samplepoints = JRC_waterClass.sample(**{
    'region': roi ,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('JRC total number of sample point:{}'.format(JRC_samplepoints.size().getInfo()))

waterclass = JRC_samplepoints.filter(ee.Filter.eq('jrcwaterclass',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))

waterclass = JRC_samplepoints.filter(ee.Filter.eq('jrcwaterclass',2))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))

In [ ]:
# 已经计算好的水体频率
# waterfrequency指已经计算好的水频率数据
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
permanent_water = waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
seasonal_water = ee.ImageCollection([waterfrequency.gte(0.25),waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
water_waterclass = ee.ImageCollection([permanent_water,seasonal_water,basemap]).sum()
# Map.addLayer(water_waterclass,{'palette':['white','red','blue'],'min':0,'max':2},"water waterclass")


JRC_sample_point = water_waterclass.sampleRegions(**{
    'collection': JRC_samplepoints,
    'properties': ['jrcwaterclass'],
    'scale': 30
})
print(JRC_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = JRC_sample_point.errorMatrix('jrcwaterclass', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

###  分两类验证

In [ ]:
JRC_2018 = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,0,1]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum().rename('jrcwaterclass')
permanentwater = waterfrequency.gte(0.75).select('frequency').rename('waterclass')
JRC_samplepoints = JRC_waterclass.sample(**{
    'region': roi ,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('JRC total number of sample point:{}'.format(JRC_samplepoints.size().getInfo()))
waterclass = JRC_samplepoints.filter(ee.Filter.eq('jrcwaterclass',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))


JRC_sample_point = permanentwater.sampleRegions(**{
    'collection': JRC_samplepoints,
    'properties': ['jrcwaterclass'],
    'scale': 30
})
print(JRC_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = JRC_sample_point.errorMatrix('jrcwaterclass', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

## Maryland

### 分三类验证

In [ ]:
# 使用Maryland做验证
Maryland_2018 = ee.Image('users/311605001111/Maryland_2018').clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland_2018.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland_2018.gte(25),Maryland_2018.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum().rename('Maryland_waterclass')
Maryland_samplepoints = Maryland_waterClass.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('Maryland total number of sample point:{}'.format(Maryland_samplepoints.size().getInfo()))

waterclass = Maryland_samplepoints.filter(ee.Filter.eq('Maryland_waterclass',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))

waterclass = Maryland_samplepoints.filter(ee.Filter.eq('Maryland_waterclass',2))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))

In [ ]:
# 已经计算好的水体频率
# waterfrequency指已经计算好的水频率数据
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
permanent_water = waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
seasonal_water = ee.ImageCollection([waterfrequency.gte(0.25),waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
water_waterclass = ee.ImageCollection([permanent_water,seasonal_water,basemap]).sum()
# Map.addLayer(water_waterclass,{'palette':['white','red','blue'],'min':0,'max':2},"water waterclass")


Maryland_sample_point = water_waterclass.sampleRegions(**{
    'collection': Maryland_samplepoints,
    'properties': ['Maryland_waterclass'],
    'scale': 30
})
print(Maryland_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = Maryland_sample_point.errorMatrix('Maryland_waterclass', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

### 分两类验证

In [ ]:
Maryland_WaterClass = Maryland_waterclass.remap([0,1,2],[0,0,1]).rename('Maryland_waterclass')
permanentwater = waterfrequency.gte(0.75).select('frequency').rename('waterclass')
Maryland_samplepoints = Maryland_WaterClass.sample(**{
    'region': roi ,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('Maryland total number of sample point:{}'.format(Maryland_samplepoints.size().getInfo()))
waterclass = Maryland_samplepoints.filter(ee.Filter.eq('Maryland_waterclass',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))


Maryland_sample_point = permanentwater.sampleRegions(**{
    'collection': Maryland_samplepoints,
    'properties': ['Maryland_waterclass'],
    'scale': 30
})
print(Maryland_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = Maryland_sample_point.errorMatrix('Maryland_waterclass', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))